In [19]:
import os
os.chdir("/home/ubuntu/code/pump_post_midterm/pump")

import json
import bnlearn as bn
import pandas as pd
import networkx as nx
import copy
from tqdm import tqdm


In [20]:
# persona_filename = 'opinions_qa/persona_val/American_Trends_Panel_W26/date0729_midterm_personas_full_sonnet.json'
persona_filename = 'opinions_qa/persona_val/American_Trends_Panel_W26/date0828_personas_American_Trends_Panel_W26_testonly_haiku.json'

with open(persona_filename, 'r') as f:
    data = json.load(f)
data[list(data.keys())[0]]

[{'name': 'Personal Values Regarding Firearms',
  'description': "The participant's core values and beliefs that shape their attitudes and behaviors towards firearm ownership, use, and safety, such as personal freedom, public safety, self-reliance, and responsible gun ownership.",
  'level': 'high',
  'candidate_values': ['values personal freedom',
   'values public safety',
   'values self-reliance and personal protection',
   'values responsible gun ownership'],
  'inferred_value': 'values public safety'},
 {'name': 'Attitudes Towards Firearms',
  'description': "The participant's overall beliefs, opinions, and feelings about guns and their role in society, including views on gun ownership, gun control, and the broader cultural attitudes towards firearms.",
  'level': 'high',
  'candidate_values': ['pro-gun', 'anti-gun', 'neutral'],
  'inferred_value': 'anti-gun'},
 {'name': 'Personality Traits',
  'description': "The participant's general disposition, including their level of anxiet

# Data Preprocessing

In [21]:
# verify data

def verify(data):
    ok_cnt = 0
    wrong_cnt = 0
    for user in data:
        for persona in data[user]:
            # if all(_ not in persona['candidate_values'] for _ in [persona['inferred_value'], persona['inferred_value'][1:-1]]):
            if persona['inferred_value'] not in persona['candidate_values']:
                wrong_cnt += 1
                print(persona)
                print(f"|{persona['inferred_value']}|")
                print(persona['candidate_values'])
                print()
            else:
                ok_cnt += 1

    print(ok_cnt, wrong_cnt)

verify(data)

{'name': 'Firearm Ownership Motivations', 'description': 'The primary reasons and purposes for owning firearms, such as hunting, self-defense, sport shooting, or collecting.', 'level': 'mid', 'candidate_values': ['hunting', 'self-defense', 'sport shooting', 'collection', 'other'], 'inferred_value': 'no guns'}
|no guns|
['hunting', 'self-defense', 'sport shooting', 'collection', 'other']

{'name': 'Perception of Crime Trends', 'description': 'How the participant perceives the overall level of crime and safety in the world compared to the past, which may shape their views on gun ownership.', 'level': 'mid', 'candidate_values': ['world is safer', 'world is more dangerous', 'world is neither safer nor more dangerous'], 'inferred_value': 'more dangerous'}
|more dangerous|
['world is safer', 'world is more dangerous', 'world is neither safer nor more dangerous']

{'name': 'Firearm Ownership Motivations', 'description': 'The primary reasons and purposes for owning firearms, such as hunting, s

In [22]:
# clean data

def clean(data):
    temp_data = copy.deepcopy(data)
    for user in data.keys():
        personas = data[user]
        clean = []
        seen_personas = []
        for p in personas:
            if f"{p['level']}/{p['name']}" in seen_personas:
                continue
            inf_val = p['inferred_value']
            cands = p['candidate_values']
            if len(cands) > 5:
                continue
            if inf_val not in cands:
                if inf_val[1:-1] in cands and all(_ in ['\'', '\"'] for _ in [inf_val[0], inf_val[-1]]):
                    p['inferred_value'] = inf_val[1:-1]
                    clean.append(p)
                    seen_personas.append(f"{p['level']}/{p['name']}")
            else:
                clean.append(p)
                seen_personas.append(f"{p['level']}/{p['name']}")
        temp_data[user] = clean
    return temp_data

cleaned_data = clean(data)
verify(cleaned_data)

38831 0


In [23]:
for user in cleaned_data:
    for idx, p in enumerate(cleaned_data[user]):
        if p['inferred_value'] not in p['candidate_values']:
            print(user, idx, p['name'], p['inferred_value'], p['candidate_values'], p)

len(set([f"{p['level']}/{p['name']}" for user in cleaned_data for p in cleaned_data[user]]))

100

In [24]:
cleaned_data['2820']

[{'name': 'Personal Values Regarding Firearms',
  'description': "The participant's core values and beliefs that shape their attitudes and behaviors towards firearm ownership, use, and safety, such as personal freedom, public safety, self-reliance, and responsible gun ownership.",
  'level': 'high',
  'candidate_values': ['values personal freedom',
   'values public safety',
   'values self-reliance and personal protection',
   'values responsible gun ownership'],
  'inferred_value': 'values public safety'},
 {'name': 'Attitudes Towards Firearms',
  'description': "The participant's overall beliefs, opinions, and feelings about guns and their role in society, including views on gun ownership, gun control, and the broader cultural attitudes towards firearms.",
  'level': 'high',
  'candidate_values': ['pro-gun', 'anti-gun', 'neutral'],
  'inferred_value': 'anti-gun'},
 {'name': 'Personality Traits',
  'description': "The participant's general disposition, including their level of anxiet

In [25]:
# categorize_data

idx2opt_mapping = {}
opt2idx_mapping = {}

for user in cleaned_data:
    for persona in cleaned_data[user]:
        p_name = f"{persona['level']}/{persona['name']}"
        if p_name in idx2opt_mapping:
            continue
        cands = persona['candidate_values']
        idx2opt = {k+1: v for k, v in enumerate(cands)}  # 0 for potential NAs
        idx2opt[0] = "Unknown"
        idx2opt_mapping[p_name] = idx2opt
        opt2idx_mapping[p_name] = {v: k for k, v in idx2opt.items()}



In [26]:
def categorize(data):
    categorized_data = copy.deepcopy(data)
    for user in data:
        personas = data[user]
        clean = []
        for idx, p in enumerate(personas):
            p_name = f"{p['level']}/{p['name']}"
            inf_val = p['inferred_value']
            try:
                p['inferred_value'] = opt2idx_mapping[p_name][inf_val]
            except:
                print(user, p_name, idx, inf_val)
                # print(p_name, p_name in opt2idx_mapping)
                # print(inf_val, inf_val in opt2idx_mapping[p_name])
                # print(opt2idx_mapping[p_name])
                print(p)
            clean.append(p)
        categorized_data[user] = clean
    return categorized_data

categorized_data = categorize(cleaned_data)

In [27]:
res = []

for user in categorized_data.keys():
    entry = {'user': user}
    for persona in categorized_data[user]:
        entry[f"{persona['level']}/{persona['name']}"] = persona['inferred_value']
    res.append(entry)

raw_df = pd.DataFrame(res)
raw_df.fillna(0, inplace=True)
raw_df = raw_df.astype(int)
df = raw_df[[_ for _ in raw_df.columns if _ is not "user"]]
df

,high/Personal Values Regarding Firearms,high/Attitudes Towards Firearms,high/Personality Traits,high/Cultural and Social Influences on Firearm Attitudes,high/Prioritization of Child Safety,high/Risk Tolerance,high/Sense of Safety,high/Socioeconomic Status,high/Worldview and Beliefs,high/Locus of Control,...,low/Health Profile,low/Firearm Storage Practices,low/Media and Online Engagement Habits,low/Firearm-Related Interests and Engagement,low/Civic Engagement on Gun Issues,low/Media Consumption Habits on Firearms,low/Perception of Firearm Availability and Violence,low/Lifestyle and Community Firearm Culture,mid/Firearm Ownership Motivations,mid/Perception of Crime Trends
0,2,2,1,3,1,2,2,2,3,2,...,2,3,3,3,2,3,2,2,0,0
1,2,2,2,3,2,2,2,2,3,2,...,2,3,3,3,2,3,2,2,0,0
2,2,2,2,2,1,2,2,1,3,2,...,2,3,3,3,2,3,2,2,2,0
3,3,1,2,1,2,1,2,2,3,2,...,2,3,3,3,2,3,2,2,2,0
4,4,3,1,3,1,2,2,2,3,2,...,2,2,3,3,2,3,2,2,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,2,2,1,3,1,2,2,2,3,2,...,2,3,3,3,2,3,2,2,0,0
390,2,2,2,3,1,2,2,1,3,2,...,2,3,3,3,2,3,2,2,2,0
391,2,2,2,3,1,2,2,2,3,2,...,1,3,3,3,2,3,2,2,0,1
392,4,3,2,3,2,2,2,2,3,2,...,1,3,3,3,2,3,2,2,0,3


# Creating BN

In [28]:

def get_bn(df, method, score):

    model = bn.structure_learning.fit(df, methodtype=method, scoretype=score, tabu_length=100)
    # Plot detected DAG
    pos = nx.spring_layout(model['adjmat'])
    # G = bn.plot(model, pos=pos)
    # dirs = []
    # for edge in G['edge_properties']:
    #     s, t = edge
    #     s = s.split('/')[0]
    #     t = t.split('/')[0]
    #     dirs.append(f"{s}_{t}")
    # from collections import Counter
    # sorted(Counter(dirs).items())


    # Compute edge strength using chi-square independence test
    model1 = bn.independence_test(model, df, alpha=0.05, prune=True)
    pos = nx.spring_layout(model['adjmat'])
    # G1 = bn.plot(model, pos=pos)
    # dirs = []
    # for edge in G1['edge_properties']:
    #     s, t = edge
    #     s = s.split('/')[0]
    #     t = t.split('/')[0]
    #     dirs.append(f"{s}_{t}")
    # from collections import Counter
    # sorted(Counter(dirs).items())


    param_model = bn.parameter_learning.fit(model1, df)

    return param_model


In [29]:
len(set(df.columns))

100

# Get new values (EM)

In [30]:
for _ in range(7):
    param_model = get_bn(df, method='hc', score='bds')
    Xtest = bn.sampling(param_model, 10)
    cols = Xtest.columns
    variables=[col for col in cols if col.startswith('mid') or col.startswith('high')]
    for var in tqdm(variables):
        Pout = bn.predict(param_model, df[[col for col in cols if col not in variables]], variables=[var])
        df[var] = Pout[var]
        raw_df[var] = Pout[var]
    
    # param_model = get_bn(df)


[bnlearn] >Computing best DAG using [hc]
[bnlearn] >Set scoring type at [bds]
[bnlearn] >Compute structure scores for model comparison (higher is better).
[bnlearn] >Compute edge strength with [chi_square]
[bnlearn] >Edge [high/Personal Values <-> mid/Emotional State] [P=0.210999] is excluded because it was not significant (P<0.05) with [chi_square]
[bnlearn] >Edge [high/Beliefs on Firearm Regulations <-> mid/Emotional Response to Gun Violence] [P=0.0923096] is excluded because it was not significant (P<0.05) with [chi_square]
[bnlearn] >Edge [mid/Concern for Child Safety <-> mid/Attitudes Towards Family Values] [P=0.4889] is excluded because it was not significant (P<0.05) with [chi_square]
[bnlearn] >Edge [mid/Concern for Child Safety <-> low/Occupation Firearm Use] [P=0.429678] is excluded because it was not significant (P<0.05) with [chi_square]
[bnlearn] >Edge [mid/Belief in Personal Responsibility <-> high/Trust in Institutions] [P=0.166565] is excluded because it was not signifi

[datazets] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[datazets] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[datazets] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[datazets] >WARNING> Probability values don't exactly sum to 1. Differ by: 1.1102230246251565e-16. Adjusting values.
[datazets] >WARNING> Probability values don't exactly sum to 1. Differ by: 1.1102230246251565e-16. Adjusting values.
[datazets] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[datazets] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[datazets] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[datazets] >WARNING> Probability values don't exactly sum to 1. 

[bnlearn] >CPD of high/Personal Values Regarding Firearms:
+--------------------------------------------+----------+
| high/Personal Values Regarding Firearms(2) | 0.335964 |
+--------------------------------------------+----------+
| high/Personal Values Regarding Firearms(3) | 0.336681 |
+--------------------------------------------+----------+
| high/Personal Values Regarding Firearms(4) | 0.327355 |
+--------------------------------------------+----------+
[bnlearn] >CPD of high/Attitudes Towards Firearms:
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+
[bnlearn] >CPD of high/Socioeconomic Status:
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+
[bnlearn] >CPD of high/Personal Values and Attitudes Towards Firearms:
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+
[bnlearn] >CPD of high/Personal Safety Concerns:
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+

  0%|          | 0/72 [00:00<?, ?it/s]

[bnlearn]> Remaining columns for inference: 15


...

 85%|████████▍ | 11/13 [00:09<00:01,  1.15it/s]

[bnlearn]> Remaining columns for inference: 15


 92%|█████████▏| 12/13 [00:10<00:00,  1.16it/s]

[bnlearn]> Remaining columns for inference: 15


100%|██████████| 13/13 [00:11<00:00,  1.16it/s]


In [31]:
raw_df

,user,high/Personal Values Regarding Firearms,high/Attitudes Towards Firearms,high/Personality Traits,high/Cultural and Social Influences on Firearm Attitudes,high/Prioritization of Child Safety,high/Risk Tolerance,high/Sense of Safety,high/Socioeconomic Status,high/Worldview and Beliefs,...,low/Health Profile,low/Firearm Storage Practices,low/Media and Online Engagement Habits,low/Firearm-Related Interests and Engagement,low/Civic Engagement on Gun Issues,low/Media Consumption Habits on Firearms,low/Perception of Firearm Availability and Violence,low/Lifestyle and Community Firearm Culture,mid/Firearm Ownership Motivations,mid/Perception of Crime Trends
0,2820,2,2,2,3,1,2,2,2,3,...,2,3,3,3,2,3,2,2,2,0
1,3718,2,2,2,3,1,2,2,2,3,...,2,3,3,3,2,3,2,2,2,0
2,362,2,2,2,3,1,2,2,2,3,...,2,3,3,3,2,3,2,2,2,0
3,1758,2,2,2,3,1,2,2,2,3,...,2,3,3,3,2,3,2,2,2,0
4,124,2,2,2,3,1,2,2,2,3,...,2,2,3,3,2,3,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,101,2,2,2,3,1,2,2,2,3,...,2,3,3,3,2,3,2,2,2,0
390,1020,2,2,2,3,1,2,2,2,3,...,2,3,3,3,2,3,2,2,2,0
391,3541,2,2,2,3,1,2,2,2,3,...,1,3,3,3,2,3,2,2,2,0
392,303,2,2,2,3,1,2,2,2,3,...,1,3,3,3,2,3,2,2,2,0


In [32]:
for var in raw_df.columns:
    if '/' not in var or var not in idx2opt_mapping: continue
    try:
        raw_df[var] = raw_df[var].apply(lambda x: idx2opt_mapping[var][x])
    except:
        print(var)
        print(idx2opt_mapping[var])

raw_df

,user,high/Personal Values Regarding Firearms,high/Attitudes Towards Firearms,high/Personality Traits,high/Cultural and Social Influences on Firearm Attitudes,high/Prioritization of Child Safety,high/Risk Tolerance,high/Sense of Safety,high/Socioeconomic Status,high/Worldview and Beliefs,...,low/Health Profile,low/Firearm Storage Practices,low/Media and Online Engagement Habits,low/Firearm-Related Interests and Engagement,low/Civic Engagement on Gun Issues,low/Media Consumption Habits on Firearms,low/Perception of Firearm Availability and Violence,low/Lifestyle and Community Firearm Culture,mid/Firearm Ownership Motivations,mid/Perception of Crime Trends
0,2820,values public safety,anti-gun,risk-taking,neutral culture,high priority on child safety,low risk tolerance,low sense of safety,high socioeconomic status,neutral worldview,...,poor health with limited coverage,no firearms kept securely,low media and online engagement,low interest and engagement,not engaged,low frequency,low perception of ease and violence,low-firearm culture,self-defense,Unknown
1,3718,values public safety,anti-gun,risk-taking,neutral culture,high priority on child safety,low risk tolerance,low sense of safety,high socioeconomic status,neutral worldview,...,poor health with limited coverage,no firearms kept securely,low media and online engagement,low interest and engagement,not engaged,low frequency,low perception of ease and violence,low-firearm culture,self-defense,Unknown
2,362,values public safety,anti-gun,risk-taking,neutral culture,high priority on child safety,low risk tolerance,low sense of safety,high socioeconomic status,neutral worldview,...,poor health with limited coverage,no firearms kept securely,low media and online engagement,low interest and engagement,not engaged,low frequency,low perception of ease and violence,low-firearm culture,self-defense,Unknown
3,1758,values public safety,anti-gun,risk-taking,neutral culture,high priority on child safety,low risk tolerance,low sense of safety,high socioeconomic status,neutral worldview,...,poor health with limited coverage,no firearms kept securely,low media and online engagement,low interest and engagement,not engaged,low frequency,low perception of ease and violence,low-firearm culture,self-defense,Unknown
4,124,values public safety,anti-gun,risk-taking,neutral culture,high priority on child safety,low risk tolerance,low sense of safety,high socioeconomic status,neutral worldview,...,poor health with limited coverage,some firearms kept securely,low media and online engagement,low interest and engagement,not engaged,low frequency,low perception of ease and violence,low-firearm culture,self-defense,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,101,values public safety,anti-gun,risk-taking,neutral culture,high priority on child safety,low risk tolerance,low sense of safety,high socioeconomic status,neutral worldview,...,poor health with limited coverage,no firearms kept securely,low media and online engagement,low interest and engagement,not engaged,low frequency,low perception of ease and violence,low-firearm culture,self-defense,Unknown
390,1020,values public safety,anti-gun,risk-taking,neutral culture,high priority on child safety,low risk tolerance,low sense of safety,high socioeconomic status,neutral worldview,...,poor health with limited coverage,no firearms kept securely,low media and online engagement,low interest and engagement,not engaged,low frequency,low perception of ease and violence,low-firearm culture,self-defense,Unknown
391,3541,values public safety,anti-gun,risk-taking,neutral culture,high priority on child safety,low risk tolerance,low sense of safety,high socioeconomic status,neutral worldview,...,good health with comprehensive coverage,no firearms kept securely,low media and online engagement,low interest and engagement,not engaged,low frequency,low perception of ease and violence,low-firearm culture,self-defense,Unknow

# Output raw_df

In [33]:

for _, row in raw_df.iterrows():
    user = str(row['user'])
    personas = categorized_data[user]
    for idx, p in enumerate(personas):
        val = row[f"{p['level']}/{p['name']}"]
        # mapping = idx2opt_mapping[f"{p['level']}/{p['name']}"]
        # new_val = mapping[val]
        # categorized_data[user][idx]['inferred_value'] = new_val
        categorized_data[user][idx]['inferred_value'] = val

new_persona_filename = 'opinions_qa/persona_val/American_Trends_Panel_W26/date0830_personas_testonly_haiku_bn_hcbds_em7.json'

with open(new_persona_filename, 'w') as f:
    json.dump(categorized_data, f, indent=4)

# Temp General

In [34]:
# import os
# os.chdir("/home/ubuntu/code/pump_post_midterm/pump")

# import json
# import bnlearn as bn
# import pandas as pd
# import networkx as nx
# from tqdm import tqdm


# for score in tqdm(['k2']):

#     persona_filename = 'opinions_qa/persona_val/American_Trends_Panel_W26/date0729_midterm_personas_full_sonnet.json'

#     with open(persona_filename, 'r') as f:
#         data = json.load(f)
#     data[list(data.keys())[0]]

#     # clean data

#     def clean(data):
#         for user in data.keys():
#             personas = data[user]
#             clean = []
#             for p in personas:
#                 inf_val = p['inferred_value']
#                 cands = p['candidate_values']
#                 if inf_val not in cands:
#                     if inf_val[1:-1] in cands and all(_ in ['\'', '\"'] for _ in [inf_val[0], inf_val[-1]]):
#                         p['inferred_value'] = inf_val[1:-1]
#                         clean.append(p)
#                 else:
#                     clean.append(p)
#             data[user] = clean

#     clean(data)

#     # categorize_data

#     idx2opt_mapping = {}
#     opt2idx_mapping = {}

#     for persona in data[list(data.keys())[0]]:
#         p_name = persona['name']
#         cands = persona['candidate_values']
#         idx2opt = {k+1: v for k, v in enumerate(cands)}  # 0 for potential NAs
#         idx2opt[0] = "Unknown"
#         idx2opt_mapping[p_name] = idx2opt
#         opt2idx_mapping[p_name] = {v: k for k, v in idx2opt.items()}

#     def categorize(data):
#         for user in data:
#             personas = data[user]
#             clean = []
#             for p in personas:
#                 p_name = p['name']
#                 inf_val = p['inferred_value']
#                 p['inferred_value'] = opt2idx_mapping[p_name][inf_val]
#                 clean.append(p)
#             data[user] = clean

#     categorize(data)



#     def get_bn(df, method, score):

#         model = bn.structure_learning.fit(df, methodtype=method, scoretype=score)
#         # Plot detected DAG
#         pos = nx.spring_layout(model['adjmat'])
#         # G = bn.plot(model, pos=pos)
#         # dirs = []
#         # for edge in G['edge_properties']:
#         #     s, t = edge
#         #     s = s.split('/')[0]
#         #     t = t.split('/')[0]
#         #     dirs.append(f"{s}_{t}")
#         # from collections import Counter
#         # sorted(Counter(dirs).items())


#         # Compute edge strength using chi-square independence test
#         model1 = bn.independence_test(model, df, alpha=0.05, prune=True)
#         pos = nx.spring_layout(model['adjmat'])
#         # G1 = bn.plot(model, pos=pos)
#         # dirs = []
#         # for edge in G1['edge_properties']:
#         #     s, t = edge
#         #     s = s.split('/')[0]
#         #     t = t.split('/')[0]
#         #     dirs.append(f"{s}_{t}")
#         # from collections import Counter
#         # sorted(Counter(dirs).items())


#         param_model = bn.parameter_learning.fit(model1, df)

#         return param_model



#     res = []
#     for user in data.keys():
#         entry = {'user': user}
#         for persona in data[user]:
#             entry[f"{persona['level']}/{persona['name']}"] = persona['inferred_value']
#         res.append(entry)

#     raw_df = pd.DataFrame(res)
#     raw_df.fillna(0, inplace=True)
#     raw_df = raw_df.astype(int)
#     df = raw_df[[_ for _ in raw_df.columns if _ != "user"]]
#     df

#     param_model = get_bn(df, method='hc', score=score)

#     for _ in range(1):
#         Xtest = bn.sampling(param_model, 10)
#         cols = Xtest.columns
#         variables=[col for col in cols if col.startswith('mid') or col.startswith('high')]
#         for var in tqdm(variables):
#             Pout = bn.predict(param_model, df[[col for col in cols if col not in variables]], variables=[var])
#             df[var] = Pout[var]
#             raw_df[var] = Pout[var]
        
#         # param_model = get_bn(df)

#     for _, row in raw_df.iterrows():
#         user = str(row['user'])
#         personas = data[user]
#         for idx, p in enumerate(personas):
#             val = row[f"{p['level']}/{p['name']}"]
#             mapping = idx2opt_mapping[p['name']]
#             new_val = mapping[val]
#             data[user][idx]['inferred_value'] = new_val

#     new_persona_filename = f'opinions_qa/persona_val/American_Trends_Panel_W26/date0826_personas_full_sonnet_bn_hc{score}.json'

#     with open(new_persona_filename, 'w') as f:
#         json.dump(data, f, indent=4)